In [1]:
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.svm import SVC
# Model Evaluation metrics 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score


In [72]:
from datetime import datetime
now = datetime.now() # current date and time
d = now.strftime("%d%m%Y")
print(d)

11072022


In [73]:
# mlbb=pd.read_csv( "hasil/2_mlbb_clean_sentiment07072022.csv",encoding='utf-8')
# ffm=pd.read_csv( "hasil/2_ffm_clean_sentiment07072022.csv",encoding='utf-8')
# hdi=pd.read_csv( "hasil/2_hdi_clean_sentiment07072022.csv",encoding='utf-8')
all=pd.read_csv( "hasil/all_game_cleancontent_08072022_noneutral.csv",encoding='utf-8')
# mlbb = mlbb[['clean_content','score','label','sentiment']]
# ffm = ffm[['clean_content','score','label','sentiment']]
# hdi = hdi[['clean_content','score','label','sentiment']]

#get sampling data random
all= all.sample(n=2000,replace=True,random_state=42)
#get sampling random 50% of data random 
# df = df.sample(n=3,replace=True,frac=0.5,random_state=42)

In [61]:
all.shape

(2000, 6)

# DATA TRAINING & TESTING SEPARATION

In [74]:
dataset=all
folder="all_rbf_c_gamma"
dtlatih=70
dtuji=100-dtlatih
# print(dtlatih,dtuji)
X = dataset['clean_content']     # Define feature matriX
y = dataset['sentiment']         # Define target feature matriX
jmluji=float(dtuji/100)
print(jmluji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=jmluji, random_state=42, stratify=y)

datalatih=('Dataset {} {}:{} \n [INFO] Sebaran kelas pada training  set: \n negative: \t {} \n neutral: \t{} \n positive:\t{} \n'.format(dataset,int((1-jmluji)*100),int(jmluji*100),sum(y_train==0), sum(y_train==1), sum(y_train==2)))
datauji=('[INFO] Sebaran kelas pada testing set:\n negative: \t{}\n neutral: \t{}\n positive:\t{} \n'.format(sum(y_test==0), sum(y_test==1), sum(y_test==2)))

dimensi_data=('[INFO] Shape Data: \n X_train: \t {} \n X_test: \t {} \n y_train: \t {} \n y_test: \t {} \n'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

with open('hasil/'+folder+'/4_1_LatihdanUji_gridsearch_'+str(dtlatih)+'-'+str(dtuji)+'_'+d+'.txt', 'a', encoding='utf-8') as f:
    f.writelines(''.join(datalatih))
    f.writelines(''.join(datauji))
    f.writelines(''.join(dimensi_data))

# print(datalatih, datauji, dimensi_data)

0.3


## SAVE DATASET

In [75]:
X_train.to_csv( "hasil/"+folder+"/4_1_"+folder+"_train_data_"+str(dtlatih)+"_"+d+".csv", index=False, encoding='utf-8-sig')
X_test.to_csv( "hasil/"+folder+"/4_1_"+folder+"_test_data_"+str(dtuji)+"_"+d+".csv", index=False, encoding='utf-8-sig')

# FEATURE EXTRACTION TFIDF

In [76]:
# ngram_range = (1,2) #bigram
ngram = (1,3) #trigram
fs="trigram"

mindf = 5

tfidf = TfidfVectorizer(ngram_range=ngram, min_df=mindf)
vectors = tfidf.fit_transform(dataset.clean_content).toarray()

In [77]:
X_train_to_df = tfidf.fit_transform(X_train).toarray()
train_words_df = pd.DataFrame(X_train_to_df, columns=tfidf.get_feature_names())
train_words_df.head(10)
train_words_df.to_csv( "hasil/"+folder+"/4_1_tfidf_training_data_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')

In [78]:
X_test_to_df = tfidf.transform(X_test).toarray()
test_words_df = pd.DataFrame(X_test_to_df, columns=tfidf.get_feature_names())
test_words_df.head(10)
test_words_df.to_csv( "hasil/"+folder+"/4_1_tfidf_testing_data_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')

# PIPELINE & GRIDSEARCH TEST

In [80]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from pprint import pprint
from time import time


In [116]:

pipe = Pipeline([
    ('bag_of_words', TfidfVectorizer(ngram_range=ngram, min_df=mindf)),
    ('estimator', SVC(random_state=42))])

#create GridSearchCV object with set of possible parameters
Cs = [0.001, 0.01, 0.1, 1, 10, 100]
gammas = [0.001, 0.01, 0.1, 1, 10,100]
kernel = ['linear','rbf','poly','sigmoid']
# kernel = ['rbf']
param_grid={
    'estimator__C': Cs,
    'estimator__gamma': gammas,
    'estimator__kernel': kernel
}

In [130]:
%%time
# scoring = {'AUCe': 'roc_auc', 'Accuracy': 'accuracy', 'prec':  'precision', 'rec': 'recall', 'f1s': 'f1','spec':make_scorer(recall_score,pos_label=0)}
# scoring=['accuracy','recall_macro','precision_macro','f1_macro']
grid = GridSearchCV(pipe, param_grid=param_grid,refit = True, verbose = 3,scoring='accuracy',n_jobs=-1)

grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)


Fitting 5 folds for each of 144 candidates, totalling 720 fits
Wall time: 30.9 s


In [110]:

# print(classification_report(, y_test )) #print classification report
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipe.steps])
print("param_grid:")
pprint(param_grid)
t0 = time()

print("done in %0.3fs" % (time() - t0))
timelapse=("Time : done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid.best_score_)
bestscore=("Best score: %0.3f" % grid.best_score_)
print("Best parameters set:")
best_parameters = grid.best_estimator_.get_params()
# print(best_parameters)
bestparam=[]
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (str(param_name), str(best_parameters[param_name])))
    bestparam=bestparam+[{'Parameter':str(param_name),'value':str(best_parameters[param_name]),'training':str(dtlatih)}]
    # bestjoin=bestjoin+"\n"+("\t%s: %r" % (param_name, best_parameters[param_name])).join(bestjoin)
    # bestparam="".join(bestparam)

Performing grid search...
pipeline: ['bag_of_words', 'estimator']
param_grid:
{'estimator__C': [0.001, 0.01, 0.1, 1, 10, 100],
 'estimator__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
 'estimator__kernel': ['rbf']}
done in 0.000s

Best score: 0.794
Best parameters set:
	estimator__C: '1'
	estimator__gamma: '1'
	estimator__kernel: 'rbf'


In [ ]:
grid.cv_results_

In [118]:
resultgs=pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["accuracy"])],axis=1)
resultgs.head()

,estimator__C,estimator__gamma,estimator__kernel,f1
0,0.001,0.001,linear,0.347623
1,0.001,0.001,rbf,0.347623
2,0.001,0.001,poly,0.347623
3,0.001,0.001,sigmoid,0.347623
4,0.001,0.010,linear,0.347623


In [119]:
bestpara=pd.DataFrame(bestparam)
bestpara.head()

,Parameter,value,training
0,estimator__C,1,70
1,estimator__gamma,1,70
2,estimator__kernel,rbf,70


In [120]:

resultgs.to_csv( "hasil/"+folder+"/4_1_resultgs_all_classifier_accuracy_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')
bestpara.to_csv( "hasil/"+folder+"/4_1_bestparameter_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')
with open('hasil/'+folder+'/4_1_gridsearch_'+folder+'_'+str(dtlatih)+'-'+str(dtuji)+'_'+fs+'_'+d+'.txt', 'a', encoding='utf-8') as f:
    f.writelines('\n')
    f.writelines((timelapse)+'\n')
    f.writelines((bestscore)+'\n')
    f.writelines('Best Parameter set:'+'\n'+(str(bestparam)))

In [121]:
grid_predictions = grid.predict(X_test)
 
# print classification report
# print(classification_report(y_test, grid_predictions,output_dict=True))
report=pd.DataFrame(classification_report(y_test, grid_predictions,output_dict=True)).T

In [122]:
print(report)
report.to_csv( "hasil/"+folder+"/4_1_testreport_gridsearch_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')

              precision    recall  f1-score     support
0              0.833828  0.878125  0.855403  320.000000
2              0.851711  0.800000  0.825046  280.000000
accuracy       0.841667  0.841667  0.841667    0.841667
macro avg      0.842769  0.839063  0.840225  600.000000
weighted avg   0.842173  0.841667  0.841237  600.000000


In [123]:
print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, grid.predict(X_test)
class_report=classification_report(y_true, y_pred)
matrix=confusion_matrix(y_true, y_pred)
print(class_report)
print()
print(matrix)
print()

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.83      0.88      0.86       320
           2       0.85      0.80      0.83       280

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600


[[281  39]
 [ 56 224]]



In [124]:
label_names = pd.Series(['negative', 'positive'])
pd.DataFrame(matrix,  
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted negative,Predicted positive
Is negative,281,39
Is positive,56,224


In [125]:
print(matrix)

[[281  39]
 [ 56 224]]


In [126]:
with open('hasil/'+folder+'/4_1_evaluasi_'+folder+'_'+str(dtlatih)+'-'+str(dtuji)+'_'+fs+'_'+d+'.txt', 'a', encoding='utf-8') as f:
    f.writelines('\n'+(matrix))

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('int64')) -> None

In [127]:
C=1
gam=1
# Create and train a random forest classifier
# forest = RandomForestClassifier()
models = [
          # ('k-NN', KNeighborsClassifier()),
          ('SVC linear kernel', SVC(kernel='linear',C=C,max_iter=10000)),
          ('SVC RBF kernel', SVC(kernel='rbf',gamma=gam,C=C)), 
          ('SVC Polynomial (degree 3)', SVC(kernel='poly',degree=3,C=C)),
          ('SVC Sigmoid ', SVC(kernel='sigmoid',C=C,gamma=gam)),
          # ('Naive Bayes', MultinomialNB()),
          # ('Decision Tree', DecisionTreeClassifier()),
          # ('Random Forest',RandomForestClassifier(n_estimators=50)),
          # ('Regresi Linier',LogisticRegression()),
        ]


In [128]:
pelatihan=[]
for name, clf in models:
  clf.fit(X_train_to_df, y_train)
  train_acc = accuracy_score(y_train, clf.predict(X_train_to_df))
  latih=('Dataset {}, data split : {}:{} \n'.format(folder,(1-jmluji)*100,jmluji*100))
  pelatihan=pelatihan+[{'classifier':name,'akurasi':train_acc,'C':C,'Gamma':gam,'training':str(dtlatih)}]
  printed_dataset=('Dataset: {} \t'.format(folder))
  printed=('[INFO] Training Menggunakan {}, akurasi pada training set: {} \n'.format(name, train_acc))
  with open('hasil/'+folder+'/4_1_training_svm_'+folder+'_'+str(dtlatih)+'-'+str(dtuji)+'_'+fs+'_'+d+'.txt', 'a', encoding='utf-8') as f:
    f.writelines('\n'+(latih))
    f.writelines(''.join(printed_dataset))
    f.writelines(''.join(printed))
  
  print(printed)
  # print(classification_report(clf.fit(X_train_to_df, y_train),y_train))

  
df_pelatihan=pd.DataFrame(pelatihan)
print(df_pelatihan)
df_pelatihan.to_csv( "hasil/"+folder+"/4_1_training_svm_"+folder+"_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')

[INFO] Training Menggunakan SVC linear kernel, akurasi pada training set: 0.8678571428571429 

[INFO] Training Menggunakan SVC RBF kernel, akurasi pada training set: 0.9285714285714286 

[INFO] Training Menggunakan SVC Polynomial (degree 3), akurasi pada training set: 0.9414285714285714 

[INFO] Training Menggunakan SVC Sigmoid , akurasi pada training set: 0.8128571428571428 

                  classifier   akurasi  C  Gamma training
0          SVC linear kernel  0.867857  1      1       70
1             SVC RBF kernel  0.928571  1      1       70
2  SVC Polynomial (degree 3)  0.941429  1      1       70
3               SVC Sigmoid   0.812857  1      1       70


In [129]:
label_names = pd.Series(['negative', 'positive'])
pengujian=[]
for name, clf in models:
  clf.predict(X_test_to_df)
  test_acc = accuracy_score(y_test, clf.predict(X_test_to_df))
  latih=('Dataset {}, data split : {}:{} \n'.format(folder,(1-jmluji)*100,jmluji*100))
  pengujian=pengujian+[{'classifier':name,'akurasi':test_acc,'C':C,'Gamma':gam,'testing':str(dtuji)}]
  printed_dataset=('Dataset: {} \t'.format(folder))
  printed=('[INFO] Testing Menggunakan {}, akurasi pada testing set: {} \n'.format(name, test_acc))
  with open('hasil/'+folder+'/4_1_testing_svm_'+folder+'_'+str(dtlatih)+'-'+str(dtuji)+'_'+fs+'_'+d+'.txt', 'a', encoding='utf-8') as f:
    f.writelines('\n'+(latih))
    f.writelines(''.join(printed_dataset))
    f.writelines(''.join(printed))
  
  print(printed)
  print(classification_report(clf.predict(X_test_to_df),y_test))
  test_matrix=(confusion_matrix(clf.predict(X_test_to_df),y_test))
  df_test=pd.DataFrame(test_matrix,  
     columns='Predicted ' + label_names,
     index='Is ' + label_names)


df_pengujian=pd.DataFrame(pengujian)
print(df_pengujian)
df_pengujian.to_csv( "hasil/"+folder+"/4_1_testing_svm_"+folder+"_"+str(dtlatih)+'-'+str(dtuji)+"_"+fs+"_"+d+".csv", index=False, encoding='utf-8-sig')

[INFO] Testing Menggunakan SVC linear kernel, akurasi pada testing set: 0.7916666666666666 

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       313
           2       0.79      0.77      0.78       287

    accuracy                           0.79       600
   macro avg       0.79      0.79      0.79       600
weighted avg       0.79      0.79      0.79       600

[INFO] Testing Menggunakan SVC RBF kernel, akurasi pada testing set: 0.8416666666666667 

              precision    recall  f1-score   support

           0       0.88      0.83      0.86       337
           2       0.80      0.85      0.83       263

    accuracy                           0.84       600
   macro avg       0.84      0.84      0.84       600
weighted avg       0.84      0.84      0.84       600

[INFO] Testing Menggunakan SVC Polynomial (degree 3), akurasi pada testing set: 0.8166666666666667 

              precision    recall  f1-score   support

       